In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoModel, AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import torch
import torch.nn as nn
import pickle

In [17]:
CRA_TOKENS =  ['[BGN]', '[END]']
tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
num_added_toks = tokenizer.add_tokens(CRA_TOKENS)
print('Added ', num_added_toks, 'tokens')

Added  2 tokens


In [18]:
model = torch.load('../results/model_CRA_5_epochs_weighted_loss.pkl')
with open(r'../data/CRA/tokenized_CRA_data_eval.pkl', "rb") as input_file:
    test_data = pickle.load(input_file)
tokens = tokenizer.convert_ids_to_tokens(test_data[0]["input_ids"])
print(tokens)
dec = tokenizer.decode(test_data[0]["input_ids"])
print(dec)

['[CLS]', 'Eget', 'företag', 'Efter', 'beslut', 'Beslutet', 'skickas', 'till', 'den', 'ambassad', 'eller', 'generalkonsul', '##at', 'som', 'du', 'valde', 'i', 'webb', '##ansökan', '.', '[BGN]', 'När', 'du', 'ska', 'hämta', 'ditt', 'beslut', 'ska', 'du', 'ta', 'med', 'ditt', 'pass', '.', '[END]', 'Du', 'kan', 'få', 'uppehåll', '##stillstånd', 'för', 'två', 'år', 'men', 'aldrig', 'längre', 'än', 'ditt', 'pass', 'är', 'giltigt', '.', 'Om', 'du', 'får', 'uppehåll', '##stillstånd', 'för', 'mer', 'än', 'tre', 'månader', 'får', 'du', 'ett', 'uppehåll', '##stillstånd', '##skort', '.', 'Kort', '##et', 'är', 'ett', 'bevis', 'på', 'att', 'du', 'har', 'tillstånd', 'att', 'vara', 'i', 'Sverige', 'och', 'innehåller', 'bland', 'annat', 'dina', 'fingeravtryck', 'och', 'foto', 'på', 'dig', '.', '[BGN]', 'Uppehåll', '##stillstånd', '##skort', '##et', 'tillverkas', 'i', 'samband', 'med', 'att', 'beslutet', 'fattas', ',', 'dock', 'tidigast', 'tre', 'månader', 'innan', 'uppehåll', '##stillståndet', 'börjar

In [19]:
def get_predicted_answers(output, tokens):
    extracted_answers = []
    last_idx = None
    current_answer = ''
    for idx, label in enumerate(output):
        if label == 2 and len(current_answer) > 0: # add to existing answer, only if there is an existing answer..
            current_answer += tokens[idx] +' '
        elif label == 1:
            # append the current answer
            if len(current_answer) > 0:
                extracted_answers.append(current_answer)
            current_answer = tokens[idx] +' '
        elif len(current_answer) > 0:
            extracted_answers.append(current_answer)
            current_answer = ''
    print(extracted_answers)




In [22]:
# Output class
# https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_tf_outputs.TFTokenClassifierOutput

model.eval()
test_input = []
test_labels = []
test_attn = []
token_type_ids = []
for i in range(len(test_data)):
    test_input.append(test_data[i]['input_ids'])
    test_labels.append(test_data[i]['labels'])
    test_attn.append(test_data[i]['attention_mask'])
    token_type_ids.append(test_data[i]['token_type_ids'])

print(len(test_input))
print(len(test_labels))
print(len(test_attn))
num_correct = 0
num_predicted = 0
num_pos_data = 0
for i in range(len(test_data)):
    output = model(torch.tensor([test_data[i]['input_ids']]), attention_mask=torch.tensor([test_data[i]['attention_mask']]), token_type_ids=torch.tensor([test_data[i]['token_type_ids']]), labels=torch.tensor([test_data[i]['labels']]))
    print('test idx: ', i)
    print('instance loss: ', output.loss)
    # print(output.logits)
    m = nn.Softmax(dim=2)
    max = m(output.logits)
    out = torch.argmax(max, dim=2)
    # print(max)
    # print('Output: ', out[0])
    # print('labels length: ', len(test_data[i]['labels']))
    # print('Labels: ', test_data[i]['labels'])
    tokens = tokenizer.convert_ids_to_tokens(test_data[i]["input_ids"])
    true_labels = test_data[i]['labels']
    # print(tokens)
    get_predicted_answers(true_labels, tokens) # print the correct answer
    get_predicted_answers(out[0], tokens)
    for idx, pred_label in enumerate(out[0]):
        true_label = true_labels[idx]
        if true_label > 0:
            num_pos_data += 1
        if pred_label > 0:
            # print('label: ', pred_label)
            # print('token: ', tokens[idx])
            num_predicted += 1
            if pred_label == true_label:
                num_correct += 1

# calculate precision and recall
pr = num_correct/num_predicted
rec = num_correct/num_pos_data
print('precision: ', pr)
print('recall: ', rec)


295
295
295
test idx:  0
instance loss:  tensor(0.6612, grad_fn=<NllLossBackward>)
['ditt pass ']
['till ', 'den ambassad eller generalkonsul ##at som du valde i ', 'webb ##ansökan ', 'När du ska hämta ditt beslut ', 'ska ', 'ta ', 'med ', 'ditt pass . ', 'två år ', 'ett ', 'dina ', 'fingeravtryck och ', 'foto på dig ', 'Uppehåll ', '##stillstånd ', '##skort ', '##et ', 'tillverkas ', 'i samband med att beslutet fattas , ', 'dock ', 'tidigast ', 'tre månader innan uppehåll ##stillståndet börjar gälla ', 'ta ', 'upp till ', 'fyra veckor ', 'att tillverka och leverera kortet ', 'till ambassaden eller generalkonsul ##atet efter att du har fått ditt beslut ']
test idx:  1
instance loss:  tensor(0.4665, grad_fn=<NllLossBackward>)
['föräldra ']
['Te ', '##ori ', 'för ', 'att ge eleverna en bred kunskap ', 'Bak ', '##ning och ', 'matlagning ', 'kunskaper ', 'närings ##lär ', 'hur ', 'livsmedels ##hantering ', 'ekonomi och ', 'lära ', 'hur ', 'olika redskap i hemmet används på ett säkert sätt 